In [1]:
import gym

_2048 = gym.make('gym_2048:2048-v0')

In [3]:
print(_2048.render())

[[0. 0. 2. 0.]
 [0. 2. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [4]:
_2048.step('left')

[array([2., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0.]),
 0,
 False,
 {}]

In [36]:
from datetime import datetime

x1 = datetime.now()

In [41]:
x1 = datetime.now()
print(datetime.now() - x1)

0:00:00.000057


# Board setup
Just use a numpy array to store the board values. 

In [34]:
board = np.zeros([4,4])
board[0,0] = 8
board[0,1] = 4
board[0,3] = 4
board[3,2] = 8

game = _2048()
game.board = board
print(game.board)
game.make_move('left')
game.board

[[8. 4. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 8. 0.]]


array([[8., 8., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 2., 0.],
       [8., 0., 0., 0.]])

In [24]:
game.make_move('left')
game.board

array([[16.,  4.,  0.,  0.],
       [ 4.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 8.,  2.,  4.,  0.]])

In [103]:
b2 = board.copy()
print(b2)
board_zero_idxs = np.where(board == 0)
random_point = np.random.choice(np.arange(board_zero_idxs[0].size))
b2[board_zero_idxs[0][random_point], board_zero_idxs[1][random_point]] = np.random.choice(np.array([2,4]))
print(b2)

[[0. 4. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 8. 0.]]
[[0. 4. 0. 4.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 8. 0.]]


# Movement
Next we need to be able to make a move where:

1. Blocks move until they hit either the edge or another non-empty square
2. Blocks in the direction of move that have the same value merge, blocks behind that are moved up
3. New 2 or 4 values are spawned in randomly

# 1. Move to left

In [20]:
board2 = np.arange(9).reshape([3,3])

#input_seq[[ix1, ix2]] = input_seq[[ix2, ix1]]
#board2[[np.where(board[:,0] == 0), 0]] = board2[[np.where(board[:,0] == 0), 1]]
board2

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [42]:
board2 = np.arange(9).reshape([3,3])
#board2[[0,0],[0,1]] = board2[[0,0],[1,0]]
    #  old_x, old_y          new_x, new_y

x=0
y=1
point_a = [0,0]
point_b = [0,1]

board2[[point_a[x], point_b[x]], [point_a[y], point_b[y]]] = board2[[point_b[x], point_a[x]],[point_b[y], point_a[y]]]

board2[0,2]

2

In [88]:
board[tuple([[0,1],0])]

array([0., 0.])

In [38]:
# Which rows in the first column are shiftable
np.where(board[:,0] == 0)
# Where this holds, the values in those rows needs to be swapped between column 0 and 1

(array([0, 1, 2, 3]),)

In [11]:
import seaborn as sns

def swap_and_merge(board, target_col_number, count):
    # Swap values
    rows_to_swap = np.where((board[:,target_col_number] == 0) & (board[:,target_col_number + 1] != 0))[0]
    target_col = np.repeat(target_col_number, rows_to_swap.size)

    if rows_to_swap.size > 0:
        board[[rows_to_swap, rows_to_swap], [target_col, target_col + 1]] = board[[rows_to_swap, rows_to_swap],[target_col + 1, target_col]]
    
    # Merge values
    rows_to_merge = np.where((board[:,target_col_number] == board[:,target_col_number + 1]) & (board[:,target_col_number] != 0))[0]
    if rows_to_merge.size > 0:
        board[tuple([rows_to_merge, target_col_number])] = 2 * board[tuple([rows_to_merge, target_col_number])]
        board[tuple([rows_to_merge, target_col_number + 1])] = 0
    
    if count == 9:                 # We've gone through the swapping so many times even the most right number has been pused all the way to the left
        return board
    elif target_col_number == 2:   # At the end of the board, back to start
        return swap_and_merge(board, 0, count + 1)
    else:                          # Next column
        return swap_and_merge(board, target_col_number + 1, count + 1)

print(board)
swap_and_merge(board.copy(), 0, 0)

[[0. 4. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 8. 0.]]


array([[8., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [8., 0., 0., 0.]])

In [26]:
def make_move(board, move):
    if move == "left":
        return swap_and_merge(board, 0, 0)
    elif move == "right":
        return np.rot90(swap_and_merge(np.rot90(board, 2), 0, 0), 2)
    elif move == 'up':
        return np.rot90(swap_and_merge(np.rot90(board, 1), 0, 0), 3)
    else:
        return np.rot90(swap_and_merge(np.rot90(board, 3), 0, 0), 1)
    
print(board)
b2 = make_move(board.copy(), 'right')
print(b2)
make_move(b2, 'down')

[[0. 4. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 8. 0.]]
[[0. 0. 0. 8.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 8.]]


array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 16.]])

In [20]:
np.rot90(board, 2)

array([[0., 8., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [4., 0., 4., 0.]])

In [73]:
x = np.zeros(5)
np.where((x == 1) | (x == 0))[0].size

5

In [36]:
input_seq = np.arange(10)
input_seq

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [37]:
ix1 = 2
ix2 = 8
input_seq[[ix1, ix2]] = input_seq[[ix2, ix1]]
input_seq

array([0, 1, 8, 3, 4, 5, 6, 7, 2, 9])